In [33]:
!pip install pymongo
!pip install pymupdf
!pip install sentence-transformers
!pip install torch --upgrade
!pip uninstall transformers sentence-transformers -y
!pip install transformers sentence-transformers
%pip install torch==1.9.0
%pip install transformers --upgrade

Found existing installation: transformers 4.49.0
Uninstalling transformers-4.49.0:
  Successfully uninstalled transformers-4.49.0
Found existing installation: sentence-transformers 3.4.1
Uninstalling sentence-transformers-3.4.1:
  Successfully uninstalled sentence-transformers-3.4.1
  Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
  Using cached sentence_transformers-3.4.1-py3-none-any.whl (275 kB)


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==1.9.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.9.0


Note: you may need to restart the kernel to use updated packages.


In [4]:
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF

In [5]:
client = MongoClient("mongodb://localhost:27017/")
print(client.list_database_names())

['admin', 'ceur_papers', 'config', 'local']


In [6]:
VOLUME_ID = "3500"  # Burayı değiştirebilirsin
BASE_URL = f"https://ceur-ws.org/Vol-{VOLUME_ID}/"

In [7]:
def get_pdf_links(volume_url):
    response = requests.get(volume_url)
    soup = BeautifulSoup(response.text, "html.parser")
    pdf_links = [BASE_URL + a["href"] for a in soup.find_all("a", href=True) if a["href"].endswith(".pdf")]
    return pdf_links

pdf_urls = get_pdf_links(BASE_URL)
print(f"{len(pdf_urls)} PDF bulundu!")

8 PDF bulundu!


In [8]:
def extract_text_from_pdf(pdf_url):
    response = requests.get(pdf_url)
    pdf_path = "temp.pdf"
    
    # PDF'yi kaydet
    with open(pdf_path, "wb") as f:
        f.write(response.content)

    # PDF'yi aç ve metni çıkar
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])

    return text

# Örnek test
pdf_url = "https://ceur-ws.org/Vol-3500/paper1.pdf"  # CEUR'dan bir PDF URL
sample_text = extract_text_from_pdf(pdf_url)
print(sample_text[:500])  # İlk 500 karakteri göster

An Ontological Modelling of Prototype Theories
Daniele Porello1, Guendalina Righetti2, Nicolas Troquard2, Roberto Confalonieri3 and
Oliver Kutz2
1University of Genoa, via Balbi 4, 16126, Genova, Italy.
2Free University of Bozen-Bolzano, piazza Domenicani 3, 39100, Bolzano, Italy
3University of Padua, Via Trieste 63, I-35121, Padova, Italy
Abstract
Prototype theories are an important family of cognitive theories of concepts that model the classification under a concept in
terms of the proximity o


In [9]:
from sentence_transformers import SentenceTransformer

# Modeli yükle
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("Model başarıyla yüklendi!")



Model başarıyla yüklendi!


In [10]:
def convert_text_to_embeddings(text):
    sentences = text.split("\n")  # Her satırı bir cümle olarak kabul edebiliriz
    embeddings = model.encode(sentences)  # Model ile embedding'e çevir
    return embeddings

In [11]:
# MongoDB'ye veri kaydetme fonksiyonu
def save_embeddings_to_mongodb(embeddings, sentences, db_name="ceur_papers", collection_name="embeddings"):
    client = MongoClient("mongodb://localhost:27017/")  # MongoDB bağlantısı
    db = client[db_name]
    collection = db[collection_name]
    
    for i, embedding in enumerate(embeddings):
        document = {
            "text": sentences[i],
            "embedding": embedding.tolist()  # Numpy array'ini listeye çeviriyoruz
        }
        collection.insert_one(document)

    print("Veriler MongoDB'ye başarıyla yüklendi!")

In [13]:
def process_pdf_and_store(pdf_url):
    # Modeli yükle
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    
    # PDF'den metin çıkar
    text = extract_text_from_pdf(pdf_url)
    
    # Metinleri embedding'e dönüştür
    embeddings = convert_text_to_embeddings(text)
    
    # MongoDB'ye kaydet
    save_embeddings_to_mongodb(embeddings, text.split("\n"))

# PDF URL'si ile işlem başlat
pdf_url = "https://ceur-ws.org/Vol-3500/paper1.pdf"  # CEUR'dan bir PDF URL
process_pdf_and_store(pdf_url)

Veriler MongoDB'ye başarıyla yüklendi!
